In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.3f}'.format


In [ ]:
import initial_condition
from process import weight


In [ ]:
global data_dir, filter_dir, output_dir
data_dir = '../../data/interim/internal/filtered_before_agg/'
filter_dir = '../../data/interim/internal/filter_after_agg/'
output_dir = '../../data/interim/internal/filtered_after_agg/'


In [ ]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE
year_range = 5

extract_population = initial_condition.EXTRACT_POPULATION
top_p_or_num = initial_condition.TOP_P_OR_NUM
top_p_or_num = ('p', 3)
region_corporation = initial_condition.REGION_CORPORATION
region_corporation = 'right_person_name'
applicant_weight = initial_condition.APPLICANT_WEIGHT

classification = initial_condition.CLASSIFICATION
# classification = 'ipc'
ipc_digit = 3
class_weight = initial_condition.CLASS_WEIGHT

filter_condition = f'{ar}_{year_style}_{extract_population}_reg_num_top_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}'
output_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'



In [ ]:
# 全体
all_df = pd.read_csv(f'{data_dir}japan.csv', 
                     encoding='utf-8', 
                     sep=',', 
                     usecols=['reg_num', 
                              region_corporation, 
                              f'{ar}_{year_style}', 
                              f'{classification}'], 
                     dtype={'reg_num': str, 
                            region_corporation: str, 
                            f'{ar}_{year_style}': np.int64, 
                            f'{classification}': str})

# all_df[f'ipc{ipc_digit}'] = all_df['ipc'].str[:ipc_digit]
all_df = all_df[all_df[f'{ar}_{year_style}'].isin(range(year_start, year_end+1))]\
    .drop_duplicates()
            #    .drop(columns=['ipc'])\
            #    .drop_duplicates()
# classification = f'ipc{ipc_dig/it}'
display(all_df.head())


# 各期間
sep_year_df_dict = {}

for year in range(year_start, year_end+1, year_range):
    sep_year_df_dict[f'{year}-{year+year_range-1}'] = all_df[all_df[f'{ar}_{year_style}'].isin(range(year, year+year_range))]


In [ ]:
all_df['right_person_name'].nunique()

In [ ]:
# 特許分類による重みづけ
# 全体
if class_weight == 'fraction':
    all_df = weight.by_classification(all_df, region_corporation, classification)
elif class_weight == 'duplication':
    all_df['class_weight'] = 1
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'


# 期間ごと
# sep_year_df_dict = {}
# sep_year_reg_num_top_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if class_weight == 'fraction':
        sep_year_df_dict[period] = weight.by_classification(sep_year_df, region_corporation, classification)
    elif class_weight == 'duplication':
        sep_year_df_dict[period] = sep_year_df.groupby([region_corporation, classification])[['reg_num']].nunique().reset_index(drop=False)
    sep_year_df_dict[period][f'{ar}_{year_style}_period'] = period

# 共同出願の重みづけ
# 全体
if applicant_weight == 'fraction':
    all_df = weight.by_applicant(all_df, region_corporation)
elif applicant_weight == 'duplication':
    all_df['applicant_weight'] = 1
all_df[f'{ar}_{year_style}_period'] = f'{year_start}-{year_end}'


# 期間ごと
# sep_year_df_dict = {}
# sep_year_reg_num_top_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if applicant_weight == 'fraction':
        sep_year_df_dict[period] = weight.by_applicant(sep_year_df, region_corporation)
    elif applicant_weight == 'duplication':
        sep_year_df_dict[period]['applicant_weight'] = 1
    sep_year_df_dict[period][f'{ar}_{year_style}_period'] = period



In [ ]:
all_reg_num_df = all_df.copy()
all_reg_num_df['reg_num'] = 1 / all_reg_num_df['class_weight'] / all_reg_num_df['applicant_weight']
all_reg_num_df = all_reg_num_df.groupby([f'{ar}_{year_style}_period', region_corporation, classification])[['reg_num']]\
                               .sum().reset_index(drop=False)\
                               .sort_values(['reg_num'], ascending=[False])
sep_year_reg_num_df_dict = sep_year_df_dict.copy()
for period, sep_year_reg_num_df in sep_year_reg_num_df_dict.items():
    sep_year_reg_num_df['reg_num'] = 1 / sep_year_reg_num_df['class_weight'] / sep_year_reg_num_df['applicant_weight']
    sep_year_reg_num_df = sep_year_reg_num_df.groupby([f'{ar}_{year_style}_period', region_corporation, classification])[['reg_num']]\
                                             .sum().reset_index(drop=False)\
                                             .sort_values(['reg_num'], ascending=[False])
    sep_year_reg_num_df_dict[period] = sep_year_reg_num_df
sep_year_reg_num_df = pd.concat([sep_year_reg_num_df for sep_year_reg_num_df in sep_year_reg_num_df_dict.values()], axis='index', ignore_index=True)
sep_year_reg_num_df


In [ ]:
# フィルタリング
reg_num_filter_df = pd.read_csv(f'{filter_dir}{filter_condition}.csv',
                                encoding='utf-8',
                                sep=',', 
                                usecols=[f'{ar}_{year_style}_period', region_corporation],
                                dtype=str)
reg_num_filter_df

In [ ]:

if extract_population == 'all':
    all_reg_num_top_df = pd.merge(
        all_reg_num_df,
        reg_num_filter_df,
        on=[f'{ar}_{year_style}_period', region_corporation],
        how='inner',
    )
    # sep_year_reg_num_top_df = pd.merge(
    #     sep_year_reg_num_df,
    #     reg_num_filter_df[[region_corporation]],
    #     on=[region_corporation], 
    #     how='inner'
    # )
    sep_year_reg_num_top_df = sep_year_reg_num_df[sep_year_reg_num_df[region_corporation].isin(reg_num_filter_df[region_corporation])]
sep_year_reg_num_top_df

reg_num_top_df = pd.concat([all_reg_num_top_df, sep_year_reg_num_top_df], 
                           axis='index', ignore_index=True)
reg_num_top_df


In [ ]:
output_condition = f'{ar}_{year_style}_{extract_population}_{top_p_or_num[0]}_{top_p_or_num[1]}_{region_corporation}_{applicant_weight}_{classification}_{class_weight}'
reg_num_top_df.to_csv(f'{output_dir}{output_condition}.csv', 
                      encoding='utf-8', 
                      sep=',', 
                      index=False)


In [ ]:
output_condition

In [ ]:
# ユニークID数を取得
total_unique_ids = len(df["ID"].unique())

# 同じ頻度のID数を集計
frequency_count = df["frequency"].value_counts().sort_index()

# 累積確率の計算
cumulative_probability = frequency_count / total_unique_ids

# 補累積確率
complementary_cumulative_probability = []
for i in range(len(cumulative_probability)):
    # 補累積確率は頻度 k+1 以降の累積確率の合計
    complementary_cumulative_probability.append(cumulative_probability[i + 1 :].sum())

def ccdf(df: pd.DataFrame) -> np.ndarray:
    freq_array = np.array(
                        np.bincount(
                                    df['frequency'].value_counts(ascending=True).to_dict().values()
                                    )
                        )
    p_list = []
    cumsum = 0.0
    s = float(freq_array.sum())
    for freq in freq_array:
        if freq != 0:
            cumsum += freq / s
            p_list.append(cumsum)
        else:
            p_list.append(1.0)
            
    ccdf_array = 1 - np.array(p_list)
    if ccdf_array[0] == 0:
        ccdf_array[0] = 1.0
    return ccdf_array